# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sqlalchemy import create_engine
import sqlite3
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')

connection = engine.connect()
query = "SELECT * FROM DisasterMessages"
df = pd.read_sql(query, connection)
connection.close()

df.shape

(26216, 39)

In [3]:
# Select X for analysis

X = df["message"].values

In [4]:
# Select Y for analysis

N_CATEGORIES = 36 #number of category columns
df_y = df.iloc[:,-N_CATEGORIES:]
df_y_sufficient_data =  df_y.loc[:,(df_y.sum(axis=0) > 50)]
Y = df_y_sufficient_data.values
N_CATEGORIES = df_y_sufficient_data.shape[1]
N_CATEGORIES # Columns with sufficient values
# df_y_sufficient_data.columns

35

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Bring to lowercase, remove stopwords and punctuation
    stop = set(stopwords.words('english') + list(string.punctuation))
    nltk_tokens = nltk.word_tokenize(text.lower())
    tokens = [w for w in nltk_tokens if not w in stop]
    return(tokens)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)
mo = MultiOutputClassifier(LogisticRegression(solver="liblinear"))
pipeline = Pipeline([('tfidf', tfidf_vectorizer), ('mo', mo)])

### 4. Train pipeline
- Split data into train and test sets


In [7]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

- Train pipeline

In [8]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001F05A1D3E50>)),
                ('mo',
                 MultiOutputClassifier(estimator=LogisticRegression(solver='liblinear')))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
Y_pred = pipeline.predict(X_validation)

Report the classification results

In [27]:
classification_result = list()

for i in range(0,N_CATEGORIES):
    # classification_result.append(classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1))
    weighted_avg = classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1,output_dict=True)['weighted avg']
    classification_result.append(weighted_avg)

classification_result

[{'precision': 0.8037287649807435,
  'recall': 0.8119755911517925,
  'f1-score': 0.7873311537708536,
  'support': 5244},
 {'precision': 0.8968208639701514,
  'recall': 0.9016018306636155,
  'f1-score': 0.8924236573324319,
  'support': 5244},
 {'precision': 0.9950665349408077,
  'recall': 0.9950419527078566,
  'f1-score': 0.9925690898928685,
  'support': 5244},
 {'precision': 0.783311520953859,
  'recall': 0.7841342486651411,
  'f1-score': 0.7807043767806096,
  'support': 5244},
 {'precision': 0.9104145114672668,
  'recall': 0.9262013729977117,
  'f1-score': 0.9047424301580069,
  'support': 5244},
 {'precision': 0.9490760140816503,
  'recall': 0.9544241037376049,
  'f1-score': 0.9388930927580632,
  'support': 5244},
 {'precision': 0.977459029523441,
  'recall': 0.9769260106788711,
  'f1-score': 0.9665647596833076,
  'support': 5244},
 {'precision': 0.983500521608801,
  'recall': 0.9832189168573608,
  'f1-score': 0.9748993721762601,
  'support': 5244},
 {'precision': 0.9591695089015733,


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = 

cv = 

SyntaxError: invalid syntax (Temp/ipykernel_14428/3277122076.py, line 1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(classification_report(y_true, y_pred))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.